In [1]:
import pandas as pd
import numpy as np
import re,pyperclip

In [2]:
repL4L="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/"
repIMM21=repL4L+"IMM21/"
fSwim=repIMM21+"IMM21-00-X-paradigmes-Swim1.csv"
fSwim=repIMM21+"IMM21-00-X-paradigmes-Swim1-StemSpace.csv"
fSwim=repIMM21+"IMM21-00-X-paradigmes-Swim2.csv"
fSwim=repIMM21+"IMM21-00-X-paradigmes-Swim2-StemSpace.csv"
fSwim=repIMM21+"IMM21-00-X-Morphomes-Swim1.csv"
fSwim=repIMM21+"IMM21-00-X-Morphomes-Swim1-StemSpace.csv"
fSwim=repIMM21+"IMM21-00-X-Morphomes-Swim2.csv"
fSwim=repIMM21+"IMM21-00-X-Morphomes-Swim2-StemSpace.csv"
# fSwim=repIMM21+"IMM21-01-X-paradigmes-Swim1.csv"
# fSwim=repIMM21+"IMM21-01-X-paradigmes-Swim1-StemSpace.csv"
# fSwim=repIMM21+"IMM21-01-X-paradigmes-Swim2.csv"
# fSwim=repIMM21+"IMM21-01-X-paradigmes-Swim2-StemSpace.csv"
# fSwim=repIMM21+"IMM21-01-X-Morphomes-Swim1.csv"
# fSwim=repIMM21+"IMM21-01-X-Morphomes-Swim1-StemSpace.csv"
# fSwim=repIMM21+"IMM21-01-X-Morphomes-Swim2.csv"
# fSwim=repIMM21+"IMM21-01-X-Morphomes-Swim2-StemSpace.csv"
fControl=repL4L+"vlexique-Sampa.csv"
# fControl=repIMM21+"IMM21-02-X-paradigmes.csv"

In [3]:
dfSwim=pd.read_csv(fSwim,sep=";",encoding="utf8")
dfControl=pd.read_csv(fControl,sep=";",encoding="utf8")

In [4]:
cols=dfControl.columns.tolist()[2:]
# cols

In [5]:
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
neutralisations=neutralisationsSUD

bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [6]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [7]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [8]:
def checkFrench(prononciation):
    glide2voyelle={"j":"i","w":"u","H":"y"}
    if prononciation:
        if "," in prononciation:
            prononciations=prononciation.split(",")
            setPrononciations=set()
            for element in prononciations:
                setPrononciations.add(checkFrench(element))
            result=",".join(list(setPrononciations))
        else:
            result=recoder(prononciation)
            result=result.replace("Jj","J")
            m=re.match(ur"^(.*[^ieèEaOouy926êôâ])([jwH])$",result)
            if m:
                print ("pb avec un glide final", prononciation)
                voyelle=glide2voyelle[m.group(2)]
                result=m.group(1)+voyelle
            m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
            if m:
                n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
                if not n:
                    glide=m.group(2)
                    result=m.group(1)+dierese[glide]+m.group(3)
            m=re.match(ur"(.*)([iuy])([ieEaOouyêâô].*)",result)
            if m:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
            result=result.replace("jj","ij")
    else:
        result=prononciation
    return result

In [9]:
print checkFrench(u"aliJje")

aliJE


In [10]:
equals={}
differents={}
missings={}

def normalise(forme):
    result=recoder(forme)
    result=checkFrench(result)
    return result        

def compare(row):
    def compareForms(f1,f2):
        if normalise(f1)==normalise(f2):
            lEquals[c]=(f1,f2)
        else:
            lDifferents[c]=(f1,f2)
        return

    
    lexeme=row.lexeme
    if lexeme in dfSwim["lexeme"].tolist():
        lEquals=equals[lexeme]={}
        lDifferents=differents[lexeme]={}
        lMissings=missings[lexeme]={}
        swimRow=dfSwim[dfSwim.lexeme.str.contains("^"+lexeme+"$")]
        for c in cols:
            swimRowC=swimRow[c].values[0]
            if row[c]==row[c]:
                if swimRowC==swimRowC:
                    compareForms(row[c],swimRowC)
                else:
                    lMissings[c]=(row[c],swimRowC)
        return (lexeme,len(lEquals),len(lDifferents),len(lMissings))
    return (lexeme,np.nan,np.nan,np.nan)

In [11]:
dfResults=pd.DataFrame([[l,e,d,m] for l,e,d,m in dfControl.apply(compare,axis=1).values], columns="lexeme equal different missing".split(" "))

In [12]:
nbForms=dfResults["equal different missing".split(" ")].sum().sum()
nbForms

214979.0

In [13]:
print fSwim.split("/")[-1],"vs",fControl.split("/")[-1]
# Paradigmes-Swim1: 43
# Paradigmes-Swim2: 92
# Morphomes-Swim1: 42
# Morphomes-Swim2: 92
ajustementsManuels=0
nbEqual=int(dfResults["equal"].sum()+ajustementsManuels)
nbDifferent=int(dfResults["different"].sum()-ajustementsManuels)
nbMissing=int(dfResults["missing"].sum())
print "formes identiques :",nbEqual,"=> %2.1f%%"%round(nbEqual/nbForms*100,1)
print "formes manquantes :",nbMissing,"=> %2.1f%%"%round(nbMissing/nbForms*100,1)
print "formes différentes :",nbDifferent,"=> %2.1f%%"%round(nbDifferent/nbForms*100,1)


IMM21-00-X-Morphomes-Swim2-StemSpace.csv vs vlexique-Sampa.csv
formes identiques : 175503 => 81.6%
formes manquantes : 37545 => 17.5%
formes différentes : 1931 => 0.9%


In [14]:
dfResults.count()

lexeme       5250
equal        4315
different    4315
missing      4315
dtype: int64

In [15]:
tMissings=[]
for k,v in missings.iteritems():
    for kk,vv in v.iteritems():
        tMissings.append("\t".join([k,kk,vv[0]]))
pyperclip.copy("\n".join(tMissings))

In [16]:
dfDiff=pd.DataFrame.from_dict(differents).T

In [17]:
nbEqualCorr=nbEqual
nbDifferentCorr=nbDifferent
nbMissingCorr=nbMissing

for k,v in differents.iteritems():
    for kk,vv in v.iteritems():
        if "," in vv[0] or "," in vv[1]:
            set1=set(vv[0].split(","))
            set2=set(vv[1].split(","))
            sEquals12=set1&set2
            equals12=",".join(sEquals12)
            sMissings12=set1-set2
            missings12=",".join(sMissings12)
            sDifferents12=set2-set1
            differents12=",".join(sDifferents12)
            # print ",".join([k,kk,"=",equals12,u"Ø",missings12,u"≠",differents12])
            nbEqualCorr+=len(sEquals12)
            nbDifferentCorr-=len(sEquals12)
            nbMissingCorr+=len(sMissings12)
print fSwim.split("/")[-1],"vs",fControl.split("/")[-1]
print "formes identiques :",nbEqualCorr,"=> %2.1f%%"%round(nbEqualCorr/nbForms*100,1)
print "formes manquantes :",nbMissingCorr,"=> %2.1f%%"%round(nbMissingCorr/nbForms*100,1)
print "formes différentes :",nbDifferentCorr,"=> %2.1f%%"%round(nbDifferentCorr/nbForms*100,1)
print "précision : %2.1f%%"%(float(nbEqualCorr)/(nbEqualCorr+nbDifferentCorr)*100)
print "rappel : %2.1f%%"%(float(nbEqualCorr)/(nbEqualCorr+nbMissingCorr)*100)


IMM21-00-X-Morphomes-Swim2-StemSpace.csv vs vlexique-Sampa.csv
formes identiques : 175665 => 81.7%
formes manquantes : 37707 => 17.5%
formes différentes : 1769 => 0.8%
précision : 99.0%
rappel : 82.3%
